<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 6
# Trois méthodes de désambiguïsation lexicale

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot possède dans la phrase respective.  Les trois méthodes sont les suivantes (elles seront détaillées par la suite) :

* Algorithme de Lesk simplifié.
* Utilisation de word2vec.
* Classification supervisée utilisant des traits lexicaux.

Les deux premières méthodes n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d'une occurrence de *interest* avec chacune des définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L'algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  La dernière méthode vise à classifier les occurrences de *interest*, les sens étant les classes, et les attributs étant les mots du contexte (apprentissage supervisé).

In [1]:
import nltk
import pandas as pd

from random import randrange, shuffle
from math import floor, ceil

## 1. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html) (il se trouve en bas de sa page web).  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez répondre brièvement aux questions suivantes :

a. Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?

b. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?

c. Est-ce qu'il y a aussi des occurrences au pluriel (*interests*) à traite ?

d. Comment sont annotées les phrases qui contiennent plusieurs occurrences du mot *interest* ?

> a.
>    - Archive du corpus: https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz
>    - Fichier README: https://www.d.umn.edu/~tpederse/Data/README.int.txt
>
> b. Les phrases sont séparées par lignes intercalées par des séparateurs `$$`, chaque phrase contient au moins une occurrence du mot *interest* annoté par un numéro de sens (1 à 6) précédé d'un underscore (e.g. `interest_6` indiquant "money paid for the use of money"). Les tokens, incluant la ponctuation, sont séparés par des espaces. Si un segment filtré d'une phrase n'est pas immédiatement adjacent au segment précédent ou suivant dans le corpus original, ils sont intercalés par un séparateur `===...`.
>
> c. Oui, les occurrence du mot au pluriel sont marquées telles quelles et annotées de la même manière (e.g. `interests_4`).
>
> d. Si une phrase contient plusieurs occurrence du mot *interest*, elles sont dupliquées avec une seule occurrence du mot taggée à la fois, l'autre étant préfixée d'un astérisque (e.g. `*interest`).

**1e.** D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait de `README`ici.

> 1.  361 occurrences (15%) - readiness to give attention
> 2.   11 occurrences (01%) - quality of causing attention to be given to
> 3.   66 occurrences (03%) - activity, etc. that one gives attention to
> 4.  178 occurrences (08%) - advantage, advancement or favor
> 5.  500 occurrences (21%) - a share in a company or business
> 6. 1252 occurrences (53%) - money paid for the use of money

**1f.** De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés en écrivant par exemple `Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"`.

> La version électronique de la première édition du [Longman's Dictionary of Contemporary English](https://www.ldoceonline.com/). La version utilisée pour les annotations n'est plus consultable. La version actuelle (la sixième édition) définit les 6 sens comme suit:
>
> 1. Sense 1 = "if you have an interest in something or someone, you want to know or learn more about them"
> 2. Sense 2 = "a quality or feature of something that attracts your attention or makes you want to know more about it"
> 3. Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"
> 4. Sense 4 = "the things that bring advantages to someone or something"
> 5. Sense 5 = "if you have an interest in a particular company or industry, you own shares in it"
> 6. Sense 6 = "the extra money that you must pay back when you borrow money", "money paid to you by a bank or financial institution when you keep money in an account there"

**1g.** En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Veuillez indiquer comme avant la **définition** de chaque synset pour chacun des six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

In [2]:
# Instructs NLTK to load the local WordNet 3.1 database, since the online version does not exist anymore.
from os import path

nltk.data.path.append(path.abspath(path.join(path.curdir, 'nltk_data')))

In [3]:
from nltk.corpus import wordnet as wn

print(f"WordNet version: {wn.get_version()}")

sn = wn.synsets('interest', pos=wn.NOUN)
print(f"Il y a {len(sn)} synsets pour le nom commun 'interest'")

df = pd.DataFrame(columns=['Synset', 'POS', 'Definition', 'Lemmas', 'Examples'])
for i, s in enumerate(sorted(sn, key=lambda x: x.name())):
  df.loc[i + 1] = [s.name(), s.pos(), s.definition(), ', '.join(s.lemma_names()), '; '.join(s.examples())]

df

WordNet version: 3.1
Il y a 7 synsets pour le nom commun 'interest'


,Synset,POS,Definition,Lemmas,Examples
1,interest.n.01,n,a sense of concern with and curiosity about so...,"interest, involvement",an interest in music
2,interest.n.03,n,the power of attracting or holding one's atten...,"interest, interestingness",they said nothing of great interest; primary c...
3,interest.n.04,n,a fixed charge for borrowing money; usually a ...,interest,how much interest do you pay on your mortgage?
4,interest.n.05,n,(law) a right or legal share of something; a f...,"interest, stake",they have interests all over the world; a stak...
5,interest.n.06,n,(usually plural) a social group whose members ...,"interest, interest_group",the iron interests stepped up production
6,pastime.n.01,n,a diversion that occupies one's time and thoug...,"pastime, interest, pursuit",sailing is her favorite pastime; his main past...
7,sake.n.01,n,a reason for wanting something done,"sake, interest",for your sake; died for the sake of his countr...


> 1. Sense 1 = Synset(interest.n.01) = "a sense of concern with and curiosity about someone or something"
> 2. Sense 2 = Synset(interest.n.03) = "the power of attracting or holding one's attention (because it is unusual or exciting etc.)"
> 3. Sense 3 = Synset(pastime.n.01) = "a diversion that occupies one's time and thoughts (usually pleasantly)"
> 4. Sense 4 = Synset(sake.n.01) = "a reason for wanting something done"
> 5. Sense 5 = Synset(interest.n.05) = "(law) a right or legal share of something; a financial involvement with something"
> 6. Sense 6 = Synset(interest.n.04) = "a fixed charge for borrowing money; usually a percentage of the amount borrowed"

**1h.** Définissez (manuellement, ou avec quelques lignes de code) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [4]:
senses1 = [
  ["readiness", "give", "attention"],
  ["quality", "causing", "attention"],
  ["activity", "attention"],
  ["advantage", "advancement", "favor"],
  ["share", "company", "business"],
  ["money", "paid"],
]

print(senses1)

[['readiness', 'give', 'attention'], ['quality', 'causing', 'attention'], ['activity', 'attention'], ['advantage', 'advancement', 'favor'], ['share', 'company', 'business'], ['money', 'paid']]


**1i.** En combinant les définitions obtenues aux points (4) et (5) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** correspondants.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [5]:
senses_ldoce = [
  ["interest"],
  ["quality", "feature", "attracts", "attention"],
  ["activity", "enjoy", "doing", "subject", "enjoy", "studying"],
  ["things", "bring", "advantages"],
  ["interest", "particular", "company", "industry"],
  ["extra", "money", "pay", "back"],
]

senses_wordnet = [
  ["sense", "concern", "curiosity"],
  ["power", "attracting", "holding", "attention", "unusual", "exciting"],
  ["diversion", "occupies", "time", "thoughts", "pleasantly"],
  ["reason", "wanting"],
  ["right", "legal", "share", "financial", "involvement"],
  ["fixed", "charge", "borrowing", "money", "percentage", "amount", "borrowed"],
]

senses2 = [
  list(set(senses_ldoce[i] + senses_wordnet[i]))
  for i in range(len(senses_ldoce))
]

print(senses2)

[['curiosity', 'sense', 'concern', 'interest'], ['attracting', 'unusual', 'power', 'attention', 'quality', 'feature', 'holding', 'exciting', 'attracts'], ['occupies', 'subject', 'diversion', 'activity', 'enjoy', 'doing', 'time', 'pleasantly', 'thoughts', 'studying'], ['bring', 'advantages', 'reason', 'wanting', 'things'], ['particular', 'company', 'share', 'interest', 'industry', 'financial', 'legal', 'right', 'involvement'], ['amount', 'fixed', 'percentage', 'money', 'charge', 'pay', 'back', 'borrowed', 'extra', 'borrowing']]


**1j.** Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Ces phrases sont-elles déjà tokenisées en mots ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en trois au hasard.

In [6]:
# Veuillez répondre ici à la question.
sentences = []

FILTER = ["$$", "===", ".", ",", "'", "`", "{", "}", "(", ")", "--"]
with open("interest-original.txt", "r", encoding="utf-8") as f:
  for line in f:
    sentence = []
    for word in line.strip().split():
      if word.strip() == "":
        continue

      if any([word.startswith(f) for f in FILTER]):
        continue

      # Le dataset est déjà tokenisé avec les tokens séparés par des espaces
      sentence.append(word)

    if len(sentence) > 0:
      sentences.append(sentence)

print("Il y a {} phrases.\nEn voici 3 au hasard :".format(len(sentences)))
for i in range(3):
  print(f"{i + 1}: {sentences[randrange(len(sentences))]}")

Il y a 2368 phrases.
En voici 3 au hasard :
1: ['the', 'interest_6', 'rate', 'on', 'short-term', 'funds', 'which', 'banks', 'borrow', 'to', 'finance', 'longer-term', 'loans', 'to', 'customers', 'was', 'sharply', 'higher', 'morgan', 'said']
2: ['money', 'funds', 'invest', 'in', 'such', 'things', 'as', 'short-term', 'treasury', 'securities', 'commercial', 'paper', 'and', 'certificates', 'of', 'deposit', 'all', 'of', 'which', 'have', 'been', 'posting', 'lower', 'interest_6', 'rates', 'since', 'last', 'spring']
3: ['two', 'big', 'stocks', 'involved', 'in', 'takeover', 'activity', 'saw', 'their', 'short', 'interest_5', 'surge']


## 2. Algorithme de Lesk simplifié

**2a.** Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne l'index du sens le plus probable (entre 1 et 6) selon l'algorithme de Lesk.  Cet algorithme choisit le sens qui a le maximum de mots en commun avec le contexte de *interest*.  Vous pouvez choisir vous-mêmes la taille de ce voisinage (`window_size`).  En cas d'égalité entre deux sens, tirer la réponse au sort.

In [7]:
def wsd_lesk(senses, sentence, words=("interest", "interests"), window_size=8):
  """
  Choisit le sens d'un mot donné selon l'algorithme de Lesk simplifié.
  :param words: le/les mots d'intérêt (e.g. interest ou interests)
  :param senses: liste de listes de mots-clés par sens
  :param sentence: phrase avec une occurrence annotée de interest ou interests
  :param window_size: taille de la fenêtre autour du mot d'intérêt
  :return: numéro du sens le plus probable (1-6)
  """
  word_index = [i for i, w in enumerate(sentence) if any([w.startswith(tw) for tw in words])][0]

  best_sense = 0
  max_overlap = 0
  context = sentence[max(0, word_index - window_size):min(len(sentence), word_index + window_size + 1)]

  # pick indices at random in case of a tie, to avoid bias on the first most common sense
  sense_indices = list(range(len(senses)))
  shuffle(sense_indices)
  for i in sense_indices:
    signature = set(senses[i])
    overlap = len(signature.intersection(context))
    if overlap > max_overlap:
      max_overlap = overlap
      best_sense = i

  return 1 + best_sense

**2b.** Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la méthode de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des mots-clés par sens, et la liste de phrases, et qui retourne le score de la méthode de similarité.  Ce score sera tout simplement le pourcentage de réponses correctes (sens trouvé identique au sens annoté).

In [8]:
def evaluate_wsd(fct_name, senses, sentences, window_size=8):
  """
  Évalue la méthode de désambiguïsation lexicale.
  :param fct_name: nom de la fonction de désambiguïsation lexicale
  :param senses: liste de listes de mots-clés par sens
  :param sentences: liste de phrases avec occurrences annotées
  :return: score de la méthode de désambiguïsation lexicale
  """
  correct = 0
  for sentence in sentences:
    for i, word in enumerate(sentence):
      if word.startswith("interest_") or word.startswith("interests_"):
        reference_score = int(word.split("_")[1])
        predicted_score = fct_name(senses, sentence, window_size=window_size)
        if reference_score == predicted_score:
          correct += 1

  return correct / len(sentences)

**2c.** En fixant au mieux la taille de la fenêtre autour de *interest*, quel est le meilleur score de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

In [9]:
def run_evaluations(fct, senses):
  """
  Évalue la méthode de désambiguïsation lexicale avec différentes tailles de fenêtres.
  :param fct: la fonction à évaluer
  :param senses: liste de listes de mots-clés par sens
  :return: score maximal observé
  """
  for i in range(20):
    scores = []
    for j in range(5):
      score = evaluate_wsd(fct, senses, sentences, window_size=i + 1)
      scores.append(score)
    print(f"Window size {i + 1}: {(sum(scores) / len(scores)) * 100:.2f}%")


In [10]:
print(f"Évaluation du score avec senses1 :")
run_evaluations(wsd_lesk, senses1)

print(f"\nÉvaluation du score avec senses2 :")
run_evaluations(wsd_lesk, senses2)

Évaluation du score avec senses1 :
Window size 1: 15.50%
Window size 2: 15.67%
Window size 3: 15.88%
Window size 4: 16.60%
Window size 5: 16.72%
Window size 6: 16.93%
Window size 7: 17.10%
Window size 8: 17.41%
Window size 9: 17.80%
Window size 10: 18.16%
Window size 11: 18.24%
Window size 12: 18.20%
Window size 13: 18.46%
Window size 14: 18.64%
Window size 15: 18.81%
Window size 16: 18.99%
Window size 17: 19.03%
Window size 18: 19.32%
Window size 19: 19.39%
Window size 20: 19.47%

Évaluation du score avec senses2 :
Window size 1: 15.29%
Window size 2: 16.22%
Window size 3: 16.17%
Window size 4: 17.61%
Window size 5: 17.78%
Window size 6: 18.47%
Window size 7: 18.75%
Window size 8: 19.12%
Window size 9: 19.82%
Window size 10: 20.32%
Window size 11: 20.60%
Window size 12: 20.68%
Window size 13: 21.03%
Window size 14: 21.39%
Window size 15: 21.52%
Window size 16: 21.59%
Window size 17: 21.70%
Window size 18: 22.09%
Window size 19: 22.26%
Window size 20: 22.46%


> Naturellement, une taille de fenêtre plus grande permet de trouver plus de mots en communs. Étonnamment, le score pour une taille de fenêtre de 1 n'est pas nul donc le contexte offre déjà avec seulement trois mots pour obtenir une indication sur le sens.
>
> Les scores de la liste `senses2` sont meilleurs ce qui est logique car cette liste contient un plus grand ensemble de mots-clés pour chaque sens.
> Une des grandes critiques de l'algorithme de Lesk que nous pouvons observer ici est qu'il est très dépendant à la liste de mots-clés du sens du dictionnaire, et ceux-cis sont souvent très courts et ne contiennent pas un vocabulaire très riche.
>
> Les meilleurs scores sont autour de 18% +/- 1% pour `senses1` et 21% +/- 1% pour `senses2`. Une taille de fenêtre entre 8 et 12 semble offrir un contexte déjà suffisant.

## 3. Utilisation de word2vec pour la similarité contexte vs. synset

**3a.** En réutilisant une partie du code de `wsd_lesk`, veuillez maintenant définir une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec** étudiée dans le labo précédent. 
* Vous pouvez chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.
* Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.
* La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*.  En cas d'égalité, tirer le sens au sort.
* Vous pouvez régler la taille du voisinage (`window_size`) par l'expérimentation.  

In [11]:
import gensim
from gensim.models import KeyedVectors
from gensim import downloader

path_to_model = downloader.load("word2vec-google-news-300", return_path=True)
wv_model = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)  # C bin format

In [12]:
def wsd_word2vec(senses, sentence, words=("interest", "interests"), window_size=8):
  """
  Choisit le sens d'un mot donné en comparant les scores de similarités du cosinus word2vec.
  :param words: le/les mots d'intérêt (e.g. interest ou interests)
  :param senses: liste de listes de mots-clés par sens
  :param sentence: phrase avec une occurrence annotée de interest ou interests
  :param window_size: taille de la fenêtre autour du mot d'intérêt
  :return: numéro du sens le plus probable (1-6)
  """
  word_index = [i for i, w in enumerate(sentence) if any([w.startswith(tw) for tw in words])][0]

  best_sense = 0
  max_similarity = 0
  context = sentence[max(0, word_index - window_size):min(len(sentence), word_index + window_size + 1)]

  # pick indices at random in case of a tie, to avoid bias on the first most common sense
  sense_indices = list(range(len(senses)))
  shuffle(sense_indices)
  for i in sense_indices:
    signature = set(senses[i])
    similarity = wv_model.n_similarity(context, signature)
    if similarity > max_similarity:
      max_similarity = similarity
      best_sense = i

  return 1 + best_sense

**3b.** Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez le score de la similarité word2vec.  Comment se compare-t-il avec le score précédent (Lesk) ?

In [13]:
print(f"Évaluation du score avec senses1 :")
run_evaluations(wsd_word2vec, senses1)

print(f"\nÉvaluation du score avec senses2 :")
run_evaluations(wsd_word2vec, senses2)

Évaluation du score avec senses1 :
Window size 1: 15.62%
Window size 2: 32.26%
Window size 3: 34.12%
Window size 4: 35.85%
Window size 5: 37.33%
Window size 6: 39.19%
Window size 7: 39.70%
Window size 8: 41.39%
Window size 9: 41.68%
Window size 10: 42.91%
Window size 11: 43.58%
Window size 12: 43.71%
Window size 13: 43.71%
Window size 14: 43.37%
Window size 15: 44.05%
Window size 16: 43.75%
Window size 17: 43.92%
Window size 18: 43.29%
Window size 19: 43.29%
Window size 20: 43.71%

Évaluation du score avec senses2 :
Window size 1: 47.68%
Window size 2: 56.63%
Window size 3: 57.94%
Window size 4: 59.54%
Window size 5: 61.82%
Window size 6: 60.52%
Window size 7: 61.95%
Window size 8: 61.49%
Window size 9: 62.25%
Window size 10: 62.67%
Window size 11: 62.67%
Window size 12: 60.73%
Window size 13: 60.77%
Window size 14: 60.56%
Window size 15: 60.43%
Window size 16: 60.26%
Window size 17: 60.81%
Window size 18: 60.68%
Window size 19: 60.47%
Window size 20: 60.64%


> Après évaluation, on peut déterminer que la taile de fenêtre qui offre un bon compromis entre performance est de nouveau entre 8 et 12.
>
> Dans cette fourchette, le score avec `senses1` est de 42% +/- 2% et avec `senses2` de 62% +/- 1%.
> On peut donc conclure que la méthode de similarité du cosinus en utilisant word2vec est déjà bien plus performante que l'algorithme de Lesk simplifié. On constate toutefois comme précédemment l'importance des mots-clés définis pour chaque sens, un extrait avec un vocabulaire plus riche offre de meilleures performances et la différence est plus significative (20% avec word2vec contre 3% pour Lesk).

## 4. Classification supervisée avec des traits lexicaux
Vous entraînerez maintenant des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le premier but sera de transformer chaque phrase en un ensemble d'attributs pour formater les données en vue des expériences de classification.

Veuillez utiliser le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  De plus, il faudra séparer les données en sous-ensembles d'entraînement et de test.

On vous propose de nommer les attributs `word-k`, ..., `word-2`, `word-1`, `word+1`, `word+2`, ..., `word+k` (fenêtre de taille `2*k` autour de *interest*).  Leurs valeurs sont les mots observés aux emplacements respectifs, ou `NONE` si la position dépasse l'étendue de la phrase.  Vous ajouterez un attribut nommé `word0` qui est l'occurrence du mot *interest* au singulier ou au pluriel.  

Pour chaque occurrence de *interest*, vous devrez donc créer la représentation suivante (où `6` est le numéro du sens, essentiel pour l'entraînement, mais à cacher lors de l'évaluation) :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```

**4a.** En partant de la liste des phrases appelée `sentences` préparée plus haut, veuillez générer la liste avec toutes les représentation, appelée `items_with_features`.  Vous pouvez vous aider du livre NLTK.

In [14]:
window_size = 2

items_with_features = []
for sentence in sentences:
  for i, word in enumerate(sentence):
    if word.startswith("interest_") or word.startswith("interests_"):
      [word, sense] = word.split("_")
      result = dict()
      for j in range(1, window_size + 1):
        result[f"word-{j}"] = sentence[i - j] if i - j >= 0 else "NONE"
        result[f"word+{j}"] = sentence[i + j] if i + j < len(sentence) else "NONE"

      result["word0"] = word
      items_with_features.append([result, int(sense)])

print(len(items_with_features))
print(items_with_features[151:154])

2368
[[{'word-1': 'investor', 'word+1': 'in', 'word-2': 'NONE', 'word+2': 'stock', 'word0': 'interest'}, 1], [{'word-1': 'western', 'word+1': 'to', 'word-2': 'the', 'word+2': 'see', 'word0': 'interest'}, 4], [{'word-1': 'of', 'word+1': 'because', 'word-2': 'expression', 'word+2': 'under', 'word0': 'interest'}, 1]]


**4b.** Veuillez séparer les données aléatoirement en 80% pour l'entraînement et 20%  pour l'évaluation.  Veuillez faire une division stratifiée : les deux sous-ensembles doivent contenir les mêmes proportions de sens que l'ensemble de départ.  Ils seront appelés `iwf_train` et `iwf_test`.

In [32]:
train_ratio = 0.8

iwf_train = []
iwf_test = []

for sense in range(1, len(senses1) + 1):
  sense_items = [i for i in items_with_features if i[1] == sense]
  shuffle(sense_items)

  cutoff = floor(len(sense_items) * train_ratio)
  iwf_train.extend(sense_items[:cutoff])
  iwf_test.extend(sense_items[cutoff:])

print(len(iwf_train), ' ', len(iwf_test))
print(iwf_test[:2], iwf_test[-2:])

1891   477
[[{'word-1': 'his', 'word+1': 'in', 'word-2': 'mention', 'word+2': 'horse', 'word0': 'interest'}, 1], [{'word-1': 'with', 'word+1': 'robert', 'word-2': 'read', 'word+2': 'tomsho', 'word0': 'interest'}, 1]] [[{'word-1': '%', 'word+1': 'rate', 'word-2': '8.08', 'word+2': '8.00', 'word0': 'interest'}, 6], [{'word-1': 'over', 'word+1': 'rates', 'word-2': 'hanging', 'word+2': 'and', 'word0': 'interest'}, 6]]


**4c.** Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_train` et la tester sur `iwf_test` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score que vous obtenez (avec la fonction `accuracy` de NLTK) sur l'ensemble de test ?  Comment se compare-t-il avec les précédents ?

In [33]:
from nltk.classify import naivebayes

classifier = naivebayes.NaiveBayesClassifier.train(iwf_train)
print(f"Précision: {nltk.classify.accuracy(classifier, iwf_test) * 100:.2f}%")

Précision: 91.19%


> En conservant une la taille de fenêtre de 2 telle que proposée, nous obtenons le meilleur score de 88% +/- 3% (en fonction de la séparation aléatoire de l'ensemble d'entrainement).
>
> Des fenêtres plus larges n'offrent pas nécessairement de meilleures performances, même parfois dégrade légèrement le score (avec une taille de 12, nous obtenons 85% +/- 3%). On peut supposer que le modèle aura tendance à faire de l'overfitting si trop d'informations sont fournies, ou alors que trop souvent les mots fournis sont `NONE` car la phrase évaluée est trop petite pour fournir autant d'informations.
> On peut donc conclure que la taille de 2 offre déjà de très bonnes performances, notamment par rapport aux méthodes précédentes.
>
> Ce score est bien plus élevé que celui de l'algorithme de Lesk (21% +/- 1%) et de la similarité word2vec (62% +/- 1%).  Il est donc évident que la classification supervisée est bien plus performante que les deux autres méthodes.

**4d.** En utilisant la fonction `show_most_informative_features()`, veuillez afficher les attributs les plus informatifs et commenter le résultat.

In [34]:
classifier.show_most_informative_features(16)

Most Informative Features
                   word0 = 'interests'         3 : 1      =     66.2 : 1.0
                  word+1 = 'in'                1 : 6      =     63.6 : 1.0
                  word-1 = 'other'             3 : 6      =     42.7 : 1.0
                  word+1 = 'of'                4 : 1      =     30.6 : 1.0
                  word+2 = 'and'               6 : 5      =     23.0 : 1.0
                  word+2 = 'the'               5 : 3      =     22.0 : 1.0
                  word+2 = 'on'                6 : 5      =     19.7 : 1.0
                  word-1 = 'in'                6 : 5      =     17.2 : 1.0
                  word-1 = 'own'               4 : 6      =     16.7 : 1.0
                  word-1 = 'and'               6 : 5      =     16.5 : 1.0
                  word-2 = 'NONE'              6 : 4      =     16.3 : 1.0
                  word-1 = '%'                 5 : 6      =     14.2 : 1.0
                  word-2 = 'company'           5 : 6      =     14.0 : 1.0

> On observe les traits les plus informatifs suivants:
> - le mot `interest` au pluriel est très informatif, ce qui est logique car une bonne partie des sens ne sont pas des noms comptables et certains sens sont souvent uniquement utilisés au pluriel (ici, le modèle s'en sert pour séparer le sens 1 (un intérêt pour une chose) du sens 3 (les intérêts pour une activité)).
> - ce que l'on considère comme des stop-words (`in`, `of`) sont quand même utilisés comme traits informatifs s'ils sont positionnés directement après le mot. Ce n'est pas surprenant car, par exemple, `in` est souvent utilisé dans le sens 1 (intérêt pour une personne ou quelque chose), le modèle s'en sert principalement pour distinguer le sens 5 (intérêts pour une entreprise) du sens 6 (argent reçu pour l'utilisation de l'argent).
> - enfin, la présence du mot `other` précédant le mot est aussi considéré informatif pour distinguer du sens 3 su sens 6, ce qui est encore une fois plutôt logique (on parle souvent d'autres intérêts pour finaliser les préfèrences d'une personne).

**4e.** On souhaite également obtenir les scores pour chaque sens.  Pour ce faire, il faut demander les prédictions une par une au classifieur (voir le [livre NLTK](https://www.nltk.org/book/ch06.html)), et comptabiliser les prédictions correctes pour chaque sens.  Vous pouvez vous inspirer de `evaluate_wsd`, et écrire une fonction `evaluate_wsd_supervised(classifier, items_with_features)`, que vous appliquerez aux donnés `iwf_test`.  Veuillez afficher ces scores.

In [35]:
def evaluate_wsd_supervised(classifier, items_with_features):
  correct = [0] * len(senses1)
  total = [0] * len(senses1)

  for item in items_with_features:
    features, reference_score = item
    predicted_score = classifier.classify(features)
    if reference_score == predicted_score:
      correct[reference_score - 1] += 1
    total[reference_score - 1] += 1

  return [(i + 1, correct[i], total[i], (correct[i] / total[i]) * 100) for i in range(len(correct))]

scores = evaluate_wsd_supervised(classifier, iwf_test)
df = pd.DataFrame(columns=["Sens", "Correct", "Total", "Score"])
df.set_index("Sens", inplace=True)
for i, score in enumerate(scores):
  df.loc[i + 1] = [score[1], score[2], score[3]]

df["Score"] = df["Score"].apply(lambda x: f"{x:.2f}%")
df

,Correct,Total,Score
Sens,,,
1,61.0,73.0,83.56%
2,0.0,3.0,0.00%
3,7.0,14.0,50.00%
4,35.0,36.0,97.22%
5,90.0,100.0,90.00%
6,242.0,251.0,96.41%


## 5. Conclusion

Veuillez recopier ci-dessous, en guise de conclusion, les scores des trois expériences réalisées, pour pouvoir les comparer d'un coup d'oeil.  Quel est le meilleur score obtenu?

In [37]:
df = pd.DataFrame(columns=["Method", "Score (senses1)", "Score (senses2)"])
df.loc[0] = ["Lesk simplifié", evaluate_wsd(wsd_lesk, senses1, sentences, 12), evaluate_wsd(wsd_lesk, senses2, sentences, 12)]
df.loc[1] = ["word2Vec", evaluate_wsd(wsd_word2vec, senses1, sentences, 10), evaluate_wsd(wsd_word2vec, senses2, sentences, 10)]
df.loc[2] = ["Classification supervisée", nltk.classify.accuracy(classifier, iwf_test), "N/A"]

for col in df.columns[1:]:
  df[col] = df[col].apply(lambda x: f"{x * 100:.2f}%" if isinstance(x, float) else x)

df

,Method,Score (senses1),Score (senses2)
0,Lesk simplifié,18.24%,20.61%
1,word2Vec,42.91%,62.67%
2,Classification supervisée,91.19%,N/A


> La classification supervisée obtient le meilleur score. Dans nos itérations multiples, nous avons pu observer un score maximal de 91.19%.

## Fin du laboratoire

Merci de nettoyer votre feuille, sauvegarder le résultat, et soumettre le *notebook* sur Cyberlearn.